In [ ]:
import os
import warnings
import torch
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from autoattack import AutoAttack

# Load model
def load_model(model_path):
    model = ResNet18()
    ckpt = torch.load(model_path)
    model.load_state_dict(ckpt)
    model.cuda()
    model.eval()
    return model

# Load data
def load_data(data_dir):
    transform_chain = transforms.Compose([transforms.ToTensor()])
    item = datasets.CIFAR100(root=data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)
    return test_loader

# Create save directory
def create_save_dir(save_dir):
    os.makedirs(save_dir, exist_ok=True)

# Get attack
def get_attack(model, norm, epsilon, log_path, version):
    adversary = AutoAttack(model, norm=norm, eps=epsilon,
                           log_path=log_path, version=version)
    return adversary

def main():
    warnings.filterwarnings("ignore")

    data_dir = './data'
    norm = 'L2'  # Linf
    epsilon = 8. / 255.
    model_path = './model-cifar-ResNet18/model-ResNet18-cifar100-best_epoch.pt'
    individual = False
    save_dir = f'./standard/AA_results_cifar100_{norm}'
    batch_size = 500
    version = 'standard'
    state_path = None

    create_save_dir(save_dir)
    log_path = os.path.join(save_dir, 'log_file.txt')

    model = load_model(model_path)
    test_loader = load_data(data_dir)
    n_ex = len(test_loader.dataset)

    # Get data and labels from test_loader
    x_test = torch.cat([x for (x, y) in test_loader], 0)
    y_test = torch.cat([y for (x, y) in test_loader], 0)
    
    adversary = get_attack(model, norm, epsilon, log_path, version)

    with torch.no_grad():
        if not individual:

            adv_complete = adversary.run_standard_evaluation(x_test[:n_ex], y_test[:n_ex],
                                                              bs=batch_size, state_path=state_path)
            
            torch.save({'adv_complete': adv_complete}, f'{save_dir}/aa_{version}_1_{adv_complete.shape[0]}_eps_{epsilon:.5f}.pth')
            
        else:

            adv_complete = adversary.run_standard_evaluation_individual(x_test[:n_ex],
                                                                        y_test[:n_ex], bs=batch_size)
            
            torch.save(adv_complete, f'{save_dir}/aa_{version}_individual_1_{n_ex}_eps_{epsilon:.5f}.pth')

if __name__ == '__main__':
    main()

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Load the original CIFAR-10 dataset
data_dir = './data'
transform_chain = transforms.Compose([transforms.ToTensor()])
cifar100 = datasets.CIFAR100(root=data_dir, train=False, transform=transform_chain, download=True)

# Define a variable for the number of images to display
num_images = 5

# Select random images from the dataset
random_indices = random.sample(range(len(cifar100)), num_images)
orig_imgs = [cifar100[i][0] for i in random_indices]

# Load the adversarial images as a PyTorch tensor
filename = './standard/AA_results_cifar100_L2/aa_standard_1_10000_eps_0.03137.pth'
data = torch.load(filename)
adv_imgs = [data['adv_complete'][i] for i in random_indices]

# Function to display images using matplotlib
def display_images(imgs, title, row=1):
    for i, img in enumerate(imgs):
        img = img.permute(1, 2, 0)  # Transpose the image from (C, H, W) to (H, W, C)
        img = (img.cpu().numpy() * 255).astype(int)

        plt.subplot(2, num_images, row * num_images + i + 1)
        plt.imshow(img)
        plt.title(title)
        plt.axis('off')

# Display original images and adversarial images side by side
plt.figure(figsize=(3 * num_images, 6))
display_images(orig_imgs, 'Original', row=0)
display_images(adv_imgs, 'Adversarial', row=1)
plt.savefig('./standard/AA_results_cifar100_L2/comparison.png', dpi=300, bbox_inches='tight')
plt.show()